# Descripción del BN monolayer

En el caso del BN monolayer estamos tratando con un cristal bidimensional del base diatomica, cuya celda unidad viene dada por:

$$
\vec a_1=a(1,0);\qquad\vec a_2=a\left(-\frac{1}{2},\frac{\sqrt{3}}{2}\right);
$$

# Construyo la matriz dinámica

## Primero  clasifico los vecinos segun su distancia

In [1]:
from sympy import *
import pandas as pd

## Parametros de la red, de la celdilla y del cristal
a=Symbol('a', real=True, positive=True)
q_x=Symbol('q_x', real=True);  q_y=Symbol('q_y', real=True)
q=Matrix([q_x,q_y])
a_1=Matrix([a,0]); a_2=Rational(1,2)*Matrix([-a,sqrt(3)*a])
R_B=Rational(1,3)*a_1+Rational(2,3)*a_2; R_N=Rational(2,3)*a_1+Rational(1,3)*a_2

## Masas de los átomos, frecuencia, ...
M_B = Symbol('M_B', real=True, positive=True) #masa de los átomos de Boro y N.
M_N = Symbol('M_N', real=True, positive=True)
omega = Symbol('omega', real=True)

## Vector R_l (vector de traslación primitivo)
def R_l(l_1,l_2):
    return l_1*a_1+l_2*a_2

## Vector de posición de los átomos del cristal (en equilibrio)
def R_alpha_l(alpha,l_1,l_2):
    if alpha == 1:
        return l_1*a_1+l_2*a_2+R_B

    elif alpha == 2:
        return l_1*a_1+l_2*a_2+R_N

    else:
        print("Error, alpha solo puede ser 1 o 2 ")

## Vector unitario que une uno de los átomo alphaprima con el átomo considerado alpha, l_1,l_2
def R_hat(alphaprima,alpha,l_1,l_2):
    if (R_alpha_l(alpha,l_1,l_2)-R_alpha_l(alphaprima,0,0)).norm()>0:
        return (R_alpha_l(alpha,l_1,l_2)-R_alpha_l(alphaprima,0,0))/(R_alpha_l(alpha,l_1,l_2)\
                                                                 -R_alpha_l(alphaprima,0,0)).norm()
    else:
        return (R_alpha_l(alpha,l_1,l_2)-R_alpha_l(alphaprima,0,0))

# Distancia entre el átomo alphaprime y su vecino alpha situado en la celdilla l_1,l_2
def distancia(alphaprime,alpha,l_1,l_2):
    return (R_alpha_l(alpha,l_1,l_2)-R_alpha_l(alphaprime,0,0)).norm()/a    
    
def fase(l_1,l_2):
    return exp(I*q.dot(R_l(l_1,l_2)))

#Genero una lista con la distancia de cada átomo a los átomos de la celdilla unidad 
def valores_atomos(l_1, l_2):
    return [(k, m, i, j, R_hat(k, m, i, j), distancia(k,m,i,j)) for k in [1,2] for m in [1,2]  \
            for i in range(-l_1,l_1+1) for j in range(-l_2,l_2+1)]

## Construyo un DataFrame de pandas con la información necesaria para identificar a los primeros, segundos, ... vecinos, según su distancia a cada uno de los átomos de la celdilla unidad
columnas = [r"$\alpha\prime$",r"$\alpha$",r"$l_1$", r"$l_2$", r"$\hat R$",'Distancia']

def lista_atomos(l_1, l_2):
    return pd.DataFrame(valores_atomos(l_1,l_2),columns=columnas).sort_values(['Distancia',r"$\alpha\prime$"], ascending=[True, True])

## Mostramos el dataframe como una tabla en formato \LaTeX.
#Atomos(4,4).head(38).to_latex(escape=False,float_format="{:0.4f}".format,index=False)
from IPython.display import display
from IPython.core.display import HTML
display(HTML(lista_atomos(2,2).head(40).to_html(index=False)))

$\alpha\prime$,$\alpha$,$l_1$,$l_2$,$\hat R$,Distancia
1,1,0,0,"[0, 0]",0
2,2,0,0,"[0, 0]",0
1,2,-1,0,"[-sqrt(3)/2, -1/2]",sqrt(3)/3
1,2,0,0,"[sqrt(3)/2, -1/2]",sqrt(3)/3
1,2,0,1,"[0, 1]",sqrt(3)/3
2,1,0,-1,"[0, -1]",sqrt(3)/3
2,1,0,0,"[-sqrt(3)/2, 1/2]",sqrt(3)/3
2,1,1,0,"[sqrt(3)/2, 1/2]",sqrt(3)/3
1,1,-1,-1,"[-1/2, -sqrt(3)/2]",1
1,1,-1,0,"[-1, 0]",1


Podem comprobar que `print(a)`

Si nos fijamos en los valores de la tabla, el átomo de B de la celdilla $\vec 0$ tiene 3 primeros vecinos situados en las direcciones $1\hat y$, $-\frac{\sqrt{3}}{2}\hat x- \frac{1}{2}\hat y$ y $\frac{\sqrt{3}}{2}\hat x- \frac{1}{2} \hat y$ respecto a el (y a una distancia de $\frac{\sqrt{3}}{3} a$) 

## Paso a contruir la matriz dinámica, partiendo de la matriz de fuerzas 

Debemos construir la matriz dinámica, ya que sus valores propios son $\omega^2$, siendo $\omega$ la frecuencia de propagación de cada uno de los modos.

Para ello necesitamos calcular las posiciones de equilibrio de los átomos de la red e identificar los primeros, segundos, terceros, ... vecinos. Una vez obtenidas las posiciones de los átomos y clasificados segun la distancia al respectivo átomo de la celdilla $\vec 0$, procedemos a calcular la contribución a la matriz dinámica de cada uno de los átomos, para lo cual, necesitamos conocer la matriz de constantes de fuerza que corresponde a la interacción de cada átomo con su n-esimo vecino.

La forma general del tensor de constantes de fuerza para la interacción de un átomo con su n-esimo vecino es de la forma cite:wirtz04_phonon_disper_graph_revis:


\begin{equation}
C_n=\begin{pmatrix}
\phi_n^r&\xi_n &0\\
-\xi_n & \phi_n^{ti} & 0 \\
0 & 0 & \phi_n^{to}
\end{pmatrix}
\end{equation}

donde el sistema de coordenadas se elige de manera que $x$ es la coordenada longitudinal (en la linea que conecta los dos átomos), $y$ la coordenada transversal en el plano y $z$ la coordenada perperndicular al plano. La estructura diagonal a bloques de la matriz refleja el echo que las vibraciones interplanares y fuera de plano estan completamnte desacopladas.

Vamos a suponer (por simplificar) que un desplazamiento longitudinal (radial, que estará contenido en el plano del cristal) o transversal (tangencial, sea en el plano o perpendicular al plano) solo genera una fuerza radial o transversal, es decir, $\xi_n=0$ tal y cómo se realiza en la referencia cite:Balkanski_2000. Esta aproximación se conoce como la aproximación 4NNFC, y se necesita considerar hasta los cuartos vecinos para dar cuenta de los resultados experimentales.

Otra aproximación al problema encontrada en la literatura sobre los fonones del grafeno es la conocida como el modelo /VFF/ (valence-force field), la cual determina los parámetros de la matriz en la ref{eq:7} intrudciendo 'constantes de muelle' que determinan el cambio en la energía potencial segun diferentes deformaciones; una introducción a esta aproximación se encuentra en el apendice de la referencia cite:aizawa90_bond_soften_monol_graph_formed. Con esta aproximación se necesitan menos parametros  (constantes de fuerza) para  obtener una calidad similar al la parametrización 4NNFC (cite:wirtz04_phonon_disper_graph_revis).

Por tanto, cómo primera aproximación vamos a considerar que el tensor de constantes de fuerza  de un átomo situado en la dirección $1\hat x + 0\hat y$ tiene la forma diagonal:

\begin{equation}
C_n=\begin{pmatrix}
\phi_n^r& 0 &0\\
0 & \phi_n^{ti} & 0 \\
0 & 0 & \phi_n^{to}
\end{pmatrix}
\end{equation}


... y por lo tanto la matriz de constantes de fuerza para los átomos reales la obtenemos rotando esta matriz.

Un punto importante es que cuando consideramos la interacción entre átomos del mismo tipo (en el caso del BN que estamos considerando es fácil discriminarlos, ya que són átomos de distintos elementos los que conforman la base), debemos tener en cuenta la contribución a la matriz dinámica del átomo que estamos considerando de la celdilla $\vec 0$. Esta contribución podemos derivarla de la condición de estabilidad, ref:falkowsky 2008

In [2]:
acos(R_hat(1,2,0,0).dot([1,0]))

pi/6

In [3]:
#Angulo que forma el átomo considerado respecto al eje x
def angulo(alphaprima,alpha,l_1,l_2):
    if R_hat(alphaprima,alpha,l_1,l_2)[1] <0:
        return -acos(R_hat(alphaprima,alpha,l_1,l_2).dot([1,0]))
    
    else:
        return acos(R_hat(alphaprima,alpha,l_1,l_2).dot([1,0]))

#Matriz unitaria de rotación para cambio de ejes coordenados (entorno al eje z)
def U(theta):
    return Matrix([[cos(theta),sin(theta),0], [-sin(theta), cos(theta),0],[0,0,1]])

#Matriz de fuerza para los primeros vecinos
phi_1r__BN,phi_1ti__BN,phi_1to__BN=symbols('phi_1r__BN,phi_1ti__BN,phi_1to__BN',real=True)
phi_1r__NB,phi_1ti__NB,phi_1to__NB=phi_1r__BN,phi_1ti__BN,phi_1to__BN
Phi_10__BN=1/sqrt(M_N*M_B)*Matrix([[phi_1r__BN,0,0],[0,phi_1ti__BN,0],[0,0,phi_1to__BN]])
Phi_10__NB=1/sqrt(M_N*M_B)*Matrix([[phi_1r__NB,0,0],[0,phi_1ti__NB,0],[0,0,phi_1to__NB]])


def Phi_1l__BN(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_10__BN*U(angulo(alphaprima,alpha,l_1,l_2))

def Phi_1l__NB(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_10__NB*U(angulo(alphaprima,alpha,l_1,l_2))

def D_1l_BN(alphaprima,alpha,l_1,l_2):
    return Phi_1l__BN(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)

def D_1l_NB(alphaprima,alpha,l_1,l_2):
    return Phi_1l__NB(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)

# Matriz de fuerza para los segundos vecinos
phi_2r__BB,phi_2ti__BB,phi_2to__BB=symbols('phi_2r__BB,phi_2ti__BB,phi_2to__BB',real=True)
Phi_20__BB=1/M_B*Matrix([[phi_2r__BB,0,0],[0,phi_2ti__BB,0],[0,0,phi_2to__BB]])
#Phi_20__BB=Matrix([[alpha_2+3/(4*d**2)*gamma_1,3*sqrt(3)/(4*d**2)*gamma_1,0],[-3*sqrt(3)/(4*d**2)*gamma_1,-9/(4*d**2)*gamma_1,0],[0,0,-3/a**2*gamma_2+1/d**2*delta]])
phi_2r__NN,phi_2ti__NN,phi_2to__NN=symbols('phi_2r__NN,phi_2ti__NN,phi_2to__NN',real=True)
Phi_20__NN=1/M_N*Matrix([[phi_2r__NN,0,0],[0,phi_2ti__NN,0],[0,0,phi_2to__NN]])

                   
def Phi_2l__BB(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_20__BB*U(angulo(alphaprima,alpha,l_1,l_2))

def Phi_2l__NN(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_20__NN*U(angulo(alphaprima,alpha,l_1,l_2))

def D_2l_BB(alphaprima,alpha,l_1,l_2):
    return Phi_2l__BB(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)

def D_2l_NN(alphaprima,alpha,l_1,l_2):
    return Phi_2l__NN(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)


#Matriz de fuerza para los terceros vecinos
phi_3r__BN,phi_3ti__BN,phi_3to__BN=symbols('phi_3r__BN,phi_3ti__BN,phi_3to__BN',real=True)
phi_3r__NB,phi_3ti__NB,phi_3to__NB=phi_3r__BN,phi_3ti__BN,phi_3to__BN
Phi_30__BN=1/sqrt(M_N*M_B)*Matrix([[phi_3r__BN,0,0],[0,phi_3ti__BN,0],[0,0,phi_3to__BN]])
Phi_30__NB=1/sqrt(M_N*M_B)*Matrix([[phi_3r__NB,0,0],[0,phi_3ti__NB,0],[0,0,phi_3to__NB]])
#Phi_30__BN=Matrix([[0,0,0],[0,0,0],[0,0,2/d**2*delta]])
#Phi_30__NB=Phi_30__BN
                   
def Phi_3l__BN(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_30__BN*U(angulo(alphaprima,alpha,l_1,l_2))

def Phi_3l__NB(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_30__NB*U(angulo(alphaprima,alpha,l_1,l_2))

def D_3l_BN(alphaprima,alpha,l_1,l_2):
    return Phi_3l__BN(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)

def D_3l_NB(alphaprima,alpha,l_1,l_2):
    return Phi_3l__NB(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)

#Matriz de fuerza para los cuartos vecinos
phi_4r__BN,phi_4ti__BN,phi_4to__BN=symbols('phi_4r__BN,phi_4ti__BN,phi_4to__BN',real=True)
phi_4r__NB,phi_4ti__NB,phi_4to__NB=phi_4r__BN,phi_4ti__BN,phi_4to__BN
Phi_40__BN=1/sqrt(M_B*M_N)*Matrix([[phi_4r__BN,0,0],[0,phi_4ti__BN,0],[0,0,phi_4to__BN]])
Phi_40__NB=1/sqrt(M_B*M_N)*Matrix([[phi_4r__NB,0,0],[0,phi_4ti__NB,0],[0,0,phi_4to__NB]])
#Phi_40__BN=Matrix([[0,0,0],[0,0,0],[0,0,-1/d**2*delta]])
#Phi_40__NB=Phi_40__BN
#phi_4to__BN, phi_4to__NB= 0, 0

def Phi_4l__BN(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_40__BN*U(angulo(alphaprima,alpha,l_1,l_2))

def Phi_4l__NB(alphaprima,alpha,l_1,l_2):
    return U(-angulo(alphaprima,alpha,l_1,l_2))*Phi_40__NB*U(angulo(alphaprima,alpha,l_1,l_2))

def D_4l_BN(alphaprima,alpha,l_1,l_2):
    return Phi_4l__BN(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)

def D_4l_NB(alphaprima,alpha,l_1,l_2):
    return Phi_4l__NB(alphaprima,alpha,l_1,l_2)*fase(l_1,l_2)

# Finalmente construimos la matriz dinámica "a capas"
# Hemos visto que para considerar hasta los cuartos vecinos es suficiente con l_1,l_2=2
                   
D1BN, D1NB, D2BB, D2NN, D3BN, D3NB, D4BN, D4NB = (zeros(3) for i in range(8))
D01BN, D01NB, D02BB, D02NN, D03BN, D03NB, D04BN, D04NB= (zeros(3) for i in range(8))
for k in [1,2]:
    for m in [1,2]:
         for i in range(-2,3):
            for j in range(-2,3):
                if (k==1) & (distancia(k,m,i,j) == sqrt(3)/3 ):
                    D1BN += D_1l_BN(k,m,i,j)
                    D01BN += Phi_1l__BN(k,m,i,j)

                elif (k==2) & (distancia(k,m,i,j) == sqrt(3)/3 ):
                    D1NB += D_1l_NB(k,m,i,j)
                    D01NB += Phi_1l__NB(k,m,i,j)
                 
                elif (k==1) & (distancia(k,m,i,j) ==1):
                    D2BB += D_2l_BB(k,m,i,j)
                    D02BB += Phi_2l__BB(k,m,i,j)

                elif (k==2) & (distancia(k,m,i,j) ==1):
                    D2NN += D_2l_NN(k,m,i,j)
                    D02NN += Phi_2l__NN(k,m,i,j)

                elif (k==1) & (distancia(k,m,i,j) ==2*sqrt(3)/3):
                    D3BN += D_3l_BN(k,m,i,j)
                    D03BN += Phi_3l__BN(k,m,i,j)

                elif (k==2) & (distancia(k,m,i,j) ==2*sqrt(3)/3):
                    D3NB += D_3l_NB(k,m,i,j)
                    D03NB += Phi_3l__NB(k,m,i,j)
                    
                elif (k==1) & (distancia(k,m,i,j) ==sqrt(21)/3):
                    D4BN += D_4l_BN(k,m,i,j)
                    D04BN += Phi_4l__NB(k,m,i,j)

                elif (k==2) & (distancia(k,m,i,j) ==sqrt(21)/3):
                    D4NB += D_4l_NB(k,m,i,j)
                    D04NB += Phi_4l__NB(k,m,i,j)
                                        
# Tenemos en cuenta la contribución a la matriz dinámica de los átomos situados en 
# la celdilla 0
D2BB=D2BB-D01BN-D02BB-D03BN
D2NN=D2NN-D01NB-D02NN-D03NB                                            

### Unas simples comprobaciones para comprobar que he definido bien los tensores de fuerza y las matrices dinámicas:
(Comparando la matriz dinámica para los primeros vecinos del boro con las obtenidas "manualmente)

Así, por ejemplo, que la matriz de fuerzas del átomo de N situado en la celdilla $\vec l = (0,0)$ tiene la expresión:

In [4]:
factor(Phi_1l__BN(1,2,0,0))

Matrix([
[               3*phi_1r__BN/(4*sqrt(M_B)*sqrt(M_N)) + phi_1ti__BN/(4*sqrt(M_B)*sqrt(M_N)), -sqrt(3)*phi_1r__BN/(4*sqrt(M_B)*sqrt(M_N)) + sqrt(3)*phi_1ti__BN/(4*sqrt(M_B)*sqrt(M_N)),                                 0],
[-sqrt(3)*phi_1r__BN/(4*sqrt(M_B)*sqrt(M_N)) + sqrt(3)*phi_1ti__BN/(4*sqrt(M_B)*sqrt(M_N)),                phi_1r__BN/(4*sqrt(M_B)*sqrt(M_N)) + 3*phi_1ti__BN/(4*sqrt(M_B)*sqrt(M_N)),                                 0],
[                                                                                        0,                                                                                         0, phi_1to__BN/(sqrt(M_B)*sqrt(M_N))]])

Y su fase (para calcular su contribución a la matriz dinámica) es:

In [5]:
fase(0,0)

1

Mientras que para el átomo situado en la celdilla $\vec l=(0,1), la fase es:

In [6]:
fase(0,1)

exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2))

Y su contribución a la matriz dinámica:

In [7]:
D_1l_BN(1,2,0,1)

Matrix([
[phi_1ti__BN*exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)),                                                                    0,                                                                     0],
[                                                                    0, phi_1r__BN*exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)),                                                                     0],
[                                                                    0,                                                                    0, phi_1to__BN*exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N))]])

Comprobamos la matriz dinámica para los primeros vecinos del boro con la obtenidas "manualmente"

In [8]:
D_1l_BN(1,2,-1,0)+D_1l_BN(1,2,0,0)+D_1l_BN(1,2,0,1)-D1BN

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

## Tratamos el movimiento fuera de plano independientemente de la vibraciones dentro de plano.
Puesto que las vibraciones en el eje z estan completamente desacopladas, es más comodo trabajar con 2 matrices independientes que con una matrix $6\times 6$

In [9]:
D1BN_xy=Matrix([D1BN[i,j] for i in range(2) for j in range(2)]).reshape(2,2)
D1BN_zz=D1BN[2,2]
D1NB_xy=(Matrix([D1NB[i,j] for i in range(2) for j in range(2)]).reshape(2,2))
D1NB_zz=D1NB[2,2]

D2BB_xy=Matrix([D2BB[i,j] for i in range(2) for j in range(2)]).reshape(2,2)
D2BB_zz=D2BB[2,2]

D2NN_xy=(Matrix([D2NN[i,j] for i in range(2) for j in range(2)]).reshape(2,2))
D2NN_zz=D2NN[2,2]

D3BN_xy=Matrix([D3BN[i,j]for i in range(2) for j in range(2)]).reshape(2,2)
D3BN_zz=D3BN[2,2]
D3NB_xy=(Matrix([D3NB[i,j] for i in range(2) for j in range(2)]).reshape(2,2))
D3NB_zz=D3NB[2,2]
D4BN_xy=Matrix([D4BN[i,j] for i in range(2) for j in range(2)]).reshape(2,2)
D4BN_zz=D4BN[2,2]
D4NB_xy=(Matrix([D4NB[i,j] for i in range(2) for j in range(2)]).reshape(2,2))
D4NB_zz=D4NB[2,2]

Dsup_xy=D2BB_xy.col_insert(2,D1BN_xy+D3BN_xy+D4BN_xy)
Dinf_xy=(D1NB_xy+D3NB_xy+D4NB_xy).col_insert(2,D2NN_xy)
D_xy=Dsup_xy.row_insert(2,Dinf_xy)
#D_zz=Matrix([[D2BB_zz,D1BN_zz+D3BN_zz+D4BN_zz], [D1NB_zz+D3NB_zz+D4BN_zz,D2NN_zz]])
D_zz=Matrix([[D2BB_zz,D1BN_zz+D3BN_zz], [D1NB_zz+D3NB_zz,D2NN_zz]])

In [10]:
D_zz[0,1]

phi_1to__BN*exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_1to__BN/(sqrt(M_B)*sqrt(M_N)) + phi_1to__BN*exp(-I*a*q_x)/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(-3*a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(-a*q_x/2 - sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N))

In [11]:
D_zz[1,0]

phi_1to__BN*exp(I*(a*q_x/2 - sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_1to__BN*exp(I*a*q_x)/(sqrt(M_B)*sqrt(M_N)) + phi_1to__BN/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(-a*q_x/2 - sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(3*a*q_x/2 - sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N))

Un simple calculo para comprobar con el resultado calculado a mano (notemos que la componente z no cambia con las rotaciones)

In [12]:
D3NB_zz

phi_3to__BN*exp(I*(-a*q_x/2 - sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(a*q_x/2 + sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N)) + phi_3to__BN*exp(I*(3*a*q_x/2 - sqrt(3)*a*q_y/2))/(sqrt(M_B)*sqrt(M_N))

## Diagonalizamos y ajustamos

### Para el punto $\Gamma (k_x=0, k_y=0)$

In [13]:
from periodictable import C, B, N, constants
u=constants.atomic_mass_constant*10**3 #para que este en CGS (y las const. de fuerza en dyn)

omega_Gamma_ZO=830 #cm-1
omega_Gamma_ZA=0

D_Gamma_zz=D_zz.subs([(q_x,0),(q_y,0)])#,(M_B,B.mass*u),(M_N,N.mass*u)])
display(D_Gamma_zz.eigenvals(multiple=True)[0])
display(D_Gamma_zz.eigenvals(multiple=True)[1])

0

-6*(phi_1to__BN + phi_3to__BN)/(sqrt(M_B)*sqrt(M_N))

Podemos observar que obtenemos los mismos valores que en Falkoswsky.

In [14]:
Eq1=Eq(D_Gamma_zz.eigenvals(multiple=True)[1],omega_Gamma_ZO**2)
Eq1

Eq(-6*(phi_1to__BN + phi_3to__BN)/(sqrt(M_B)*sqrt(M_N)), 688900)

## En el punto $K \left(q_x=4\pi/(3a), q_y=0\right)$

In [17]:
omega_K_ZO=605 #cm-1
omega_K_ZA=322
D_K_zz=D_zz.subs([(q_x,4*pi/3),(a,1),(q_y,0)])
#D_K_zz=D_zz.subs([(q_x,Rational(4,3)*pi),(a,1),(q_y,0)])
#display(solve(Eq(det(D_K_zz)-omega**2,0), omega**2)[0])
#,(M_B,B.mass*u),(M_N,N.mass*u)])
#P_K_zz,D_K_zz_diagonalitzada=D_K_zz.diagonalize()
#Eq5=Eq(D_K_zz.eigenvals(multiple=True)[1]+D_K_zz.eigenvals(multiple=True)[0],omega_K_ZO**2+omega_K_ZA**2) 
#Eq6=Eq(D_K_zz.eigenvals(multiple=True)[1]-D_K_zz.eigenvals(multiple=True)[0],omega_K_ZO**2-omega_K_ZA**2)
#display(D_K_zz.eigenvals(multiple=True,simplify=True,rational=True)[0])
#display(solve(Eq(det(D_K_zz-omega**2*eye(2)).subs((-1)**Rational(2,3),-1).simplify(),0),omega**2)[0].simplify())
#display(D_K_zz.eigenvals()[0])

In [18]:
display(D_K_zz.eigenvals(multiple=True)[0])

-3*phi_2to__NN/M_N - 3*(-1)**(1/3)*phi_2to__NN/(2*M_N) + 3*(-1)**(2/3)*phi_2to__NN/(2*M_N) - 3*phi_2to__BB/M_B - 3*(-1)**(1/3)*phi_2to__BB/(2*M_B) + 3*(-1)**(2/3)*phi_2to__BB/(2*M_B) - sqrt(3)*sqrt(18*M_B**2*phi_2to__NN**2 - 9*(-1)**(2/3)*M_B**2*phi_2to__NN**2 + 9*(-1)**(1/3)*M_B**2*phi_2to__NN**2 + 4*M_B*M_N*phi_1to__BN**2 - 4*(-1)**(1/3)*M_B*M_N*phi_1to__BN**2 + 4*(-1)**(2/3)*M_B*M_N*phi_1to__BN**2 + 8*M_B*M_N*phi_1to__BN*phi_3to__BN - 8*(-1)**(1/3)*M_B*M_N*phi_1to__BN*phi_3to__BN + 8*(-1)**(2/3)*M_B*M_N*phi_1to__BN*phi_3to__BN - 36*M_B*M_N*phi_2to__BB*phi_2to__NN - 18*(-1)**(1/3)*M_B*M_N*phi_2to__BB*phi_2to__NN + 18*(-1)**(2/3)*M_B*M_N*phi_2to__BB*phi_2to__NN + 4*M_B*M_N*phi_3to__BN**2 - 4*(-1)**(1/3)*M_B*M_N*phi_3to__BN**2 + 4*(-1)**(2/3)*M_B*M_N*phi_3to__BN**2 + 18*M_N**2*phi_2to__BB**2 - 9*(-1)**(2/3)*M_N**2*phi_2to__BB**2 + 9*(-1)**(1/3)*M_N**2*phi_2to__BB**2)/(2*M_B*M_N) - 3*phi_1to__BN/(sqrt(M_B)*sqrt(M_N)) - 3*phi_3to__BN/(sqrt(M_B)*sqrt(M_N))

In [17]:
ecuacio=Eq(det(D_zz.expand().subs([(q_x,4*pi/3),(a,1),(q_y,0)])-omega**2*eye(2)),0)

In [18]:
solve(ecuacio, omega**2)

[-3*phi_2to__NN/M_N - 3*(-1)**(1/3)*phi_2to__NN/(2*M_N) + 3*(-1)**(2/3)*phi_2to__NN/(2*M_N) - 3*phi_2to__BB/M_B - 3*(-1)**(1/3)*phi_2to__BB/(2*M_B) + 3*(-1)**(2/3)*phi_2to__BB/(2*M_B) - (-1)**(2/3)*sqrt(3)*sqrt(-9*M_B**2*phi_2to__NN**2 + 9*(-1)**(1/3)*M_B**2*phi_2to__NN**2 + 18*(-1)**(2/3)*M_B**2*phi_2to__NN**2 + 4*M_B*M_N*phi_1to__BN**2 - 4*(-1)**(1/3)*M_B*M_N*phi_1to__BN**2 + 4*(-1)**(2/3)*M_B*M_N*phi_1to__BN**2 + 8*M_B*M_N*phi_1to__BN*phi_3to__BN - 8*(-1)**(1/3)*M_B*M_N*phi_1to__BN*phi_3to__BN + 8*(-1)**(2/3)*M_B*M_N*phi_1to__BN*phi_3to__BN + 18*M_B*M_N*phi_2to__BB*phi_2to__NN - 36*(-1)**(2/3)*M_B*M_N*phi_2to__BB*phi_2to__NN - 18*(-1)**(1/3)*M_B*M_N*phi_2to__BB*phi_2to__NN + 4*M_B*M_N*phi_3to__BN**2 - 4*(-1)**(1/3)*M_B*M_N*phi_3to__BN**2 + 4*(-1)**(2/3)*M_B*M_N*phi_3to__BN**2 - 9*M_N**2*phi_2to__BB**2 + 9*(-1)**(1/3)*M_N**2*phi_2to__BB**2 + 18*(-1)**(2/3)*M_N**2*phi_2to__BB**2)/(2*M_B*M_N) - 3*phi_1to__BN/(sqrt(M_B)*sqrt(M_N)) - 3*phi_3to__BN/(sqrt(M_B)*sqrt(M_N)),
 -3*phi_2to__NN/M

In [19]:
omega_M_ZO=635 #cm-1
omega_M_ZA=314

D_M_zz=D_zz.subs([(q_x,pi/a),(q_y,pi/(sqrt(3)*a))])#,(M_B,B.mass*u),(M_N,N.mass*u)])
P_M_zz,D_M_zz_diagonalitzada=D_M_zz.diagonalize()
#Eq3= Eq(D_M_zz_diagonalitzada[1,1]+D_M_zz_diagonalitzada[0,0],omega_M_ZO**2+omega_M_ZA**2)
#Eq4= Eq(D_M_zz_diagonalitzada[1,1]-D_M_zz_diagonalitzada[0,0],omega_M_ZO**2-omega_M_ZA**2)
#Eq3=Eq(D_M_zz.eigenvals(multiple=True)[0]+D_M_zz.eigenvals(multiple=True)[1],omega_M_ZO**2+omega_M_ZA**2)
#Eq4=Eq(D_M_zz.eigenvals(multiple=True)[0]-D_M_zz.eigenvals(multiple=True)[1],omega_M_ZO**2-omega_M_ZA**2)
D_M_zz_diagonalitzada[0,0].simplify()

-4*phi_2to__NN/M_N - 4*phi_2to__BB/M_B - sqrt(16*M_B**2*phi_2to__NN**2 + M_B*M_N*phi_1to__BN**2 - 6*M_B*M_N*phi_1to__BN*phi_3to__BN - 32*M_B*M_N*phi_2to__BB*phi_2to__NN + 9*M_B*M_N*phi_3to__BN**2 + 16*M_N**2*phi_2to__BB**2)/(M_B*M_N) - 3*phi_1to__BN/(sqrt(M_B)*sqrt(M_N)) - 3*phi_3to__BN/(sqrt(M_B)*sqrt(M_N))

In [20]:
((D_M_zz_diagonalitzada[0,0].simplify()-D_M_zz_diagonalitzada[1,1].simplify())**2).simplify()

64*phi_2to__NN**2/M_N**2 + 4*phi_1to__BN**2/(M_B*M_N) - 24*phi_1to__BN*phi_3to__BN/(M_B*M_N) - 128*phi_2to__BB*phi_2to__NN/(M_B*M_N) + 36*phi_3to__BN**2/(M_B*M_N) + 64*phi_2to__BB**2/M_B**2

In [21]:
Eq6=Eq((D_M_zz.eigenvals(multiple=True)[1]+D_M_zz.eigenvals(multiple=True)[0]).simplify(), omega_M_ZO**2 + omega_M_ZA**2)
Eq6

Eq(-8*phi_2to__NN/M_N - 8*phi_2to__BB/M_B - 6*phi_1to__BN/(sqrt(M_B)*sqrt(M_N)) - 6*phi_3to__BN/(sqrt(M_B)*sqrt(M_N)), 501821)

In [22]:
Eq7=Eq(((D_M_zz.eigenvals(multiple=True)[1]-D_M_zz.eigenvals(multiple=True)[0])**2).simplify(), (omega_M_ZO**2 + omega_M_ZA**2)**2)
Eq7

Eq(64*phi_2to__NN**2/M_N**2 + 4*phi_1to__BN**2/(M_B*M_N) - 24*phi_1to__BN*phi_3to__BN/(M_B*M_N) - 128*phi_2to__BB*phi_2to__NN/(M_B*M_N) + 36*phi_3to__BN**2/(M_B*M_N) + 64*phi_2to__BB**2/M_B**2, 251824316041)

In [23]:
D_M_zz.eigenvals(multiple=True)[0]

-4*phi_2to__NN/M_N - 4*phi_2to__BB/M_B + sqrt(16*M_B**2*phi_2to__NN**2 + M_B*M_N*phi_1to__BN**2 - 6*M_B*M_N*phi_1to__BN*phi_3to__BN - 32*M_B*M_N*phi_2to__BB*phi_2to__NN + 9*M_B*M_N*phi_3to__BN**2 + 16*M_N**2*phi_2to__BB**2)/(M_B*M_N) - 3*phi_1to__BN/(sqrt(M_B)*sqrt(M_N)) - 3*phi_3to__BN/(sqrt(M_B)*sqrt(M_N))

In [24]:
simplify((Eq1,Eq2,Eq6))

NameError: name 'Eq2' is not defined